In [7]:
import psycopg2
import tkinter as tk
from tkinter import ttk
import math

def connect_to_db(dbname, user, password, host="localhost", port="5432"):
    try:
        conn = psycopg2.connect(dbname="booking", user="postgres", password="123124125", host=host, port=port)
        return conn
    except psycopg2.Error as e:
        print(f"Ошибка подключения: {e}")
        return None

def execute_query(conn, query):
    try:
        cur = conn.cursor()
        cur.execute(query)
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description] if cur.description else [] 
        cur.close()
        return results, columns 
    except psycopg2.Error as e:
        print(f"Ошибка выполнения запроса: {e}")
        return None, None 

def run_custom_query():
    custom_query = custom_query_entry.get("1.0", tk.END).strip() 
    if custom_query:
        try:
            run_query(custom_query) 
        except psycopg2.Error as e:
            show_error(f"Ошибка выполнения запроса: {e}")
    else:
        show_error("Введите запрос.")
        
def show_selected_table():
    selected_table = table_combobox.get()
    if selected_table:
        query = f"SELECT * FROM {selected_table} ORDER BY {selected_table}_id;"
        run_query(query)
    else:
        show_error("Выберите таблицу.")
        
def show_error(message):
    error_label.config(text=message)
    error_label.update() 
    
def run_query(query):
    try:
        dbname = "booking"
        user = "postgres"
        password = "123124125"
        host = "localhost"
        port = "5432"

        conn = connect_to_db(dbname, user, password, host, port)
        if conn:
            results, columns = execute_query(conn, query)
            if results is not None:
                tree.delete(*tree.get_children())
                if columns:
                    tree["columns"] = columns
                    for col in columns:
                        tree.heading(col, text=col)
                    for row in results:
                        tree.insert("", "end", values=row)
                else:
                    tree.insert("", "end", values=("Запрос выполнен успешно (без результатов).",))
            conn.close()
            tree.update()
            error_label.config(text="")
    except Exception as e:
        show_error(f"Произошла неизвестная ошибка: {e}")

root = tk.Tk()
root.title("PostgreSQL Interface")

table_label = tk.Label(root, text="Выберите таблицу:")
table_label.place(x=10,y=10)
table_combobox = ttk.Combobox(root)
table_combobox.place(x=10,y=40)
table_combobox['values'] = [("client"),("house"),("booking"),("discount"),("extra_service"),("payment"),("feedback")]

button0 = tk.Button(root, text="Показать таблицу", command=show_selected_table)
button0.place(x=10, y=70)

custom_query_label = tk.Label(root, text="Введите произвольный запрос:")
custom_query_label.place(x=10,y=110)
custom_query_entry = tk.Text(root, height=5, width=40) 
custom_query_entry.place(x=10,y=140)

button_custom_query = tk.Button(root, text="Выполнить запрос", command=run_custom_query)
button_custom_query.place(x=10,y=250)
    
query_list = [
    ("Запрос 1","SELECT * FROM house WHERE cost_per_day < 6500 AND (capacity = 4 OR capacity = 5) ORDER BY house_id;"),
    ("Запрос 2","SELECT booking_id, start_date, end_date FROM booking WHERE end_date BETWEEN '2024-11-21' AND '2025-01-01' ORDER BY booking_id;"),
    ("Запрос 3","SELECT house_id, AVG(rating) FROM feedback GROUP BY house_id ORDER BY AVG(rating) DESC LIMIT 8;"),
    ("Запрос 4","SELECT booking_id, overall_cost, CONCAT(overall_cost, ' рублей 00 копеек') AS formatted_cost FROM booking ORDER BY booking_id;"),
    ("Запрос 5","""SELECT house.house_id, COUNT(booking.booking_id) AS times_booked FROM house 
LEFT JOIN booking ON house.house_id = booking.house_id GROUP BY house.house_id ORDER BY house.house_id;"""),
    ("Запрос 6","""SELECT h1.house_id AS h1_house_id, h1.capacity AS h1_capacity, h2.house_id AS h2_house_id, h2.capacity AS h2_capacity 
FROM house h1 JOIN house h2 ON h1.capacity = h2.capacity WHERE h1.house_id != h2.house_id;"""),
    ("Запрос 7","SELECT h.house_id, h.capacity, h.cost_per_day FROM house h WHERE h.capacity = (SELECT MIN(capacity) FROM house);"),
    ("Запрос 8","""SELECT b.booking_id, b.start_date, b.overall_cost FROM booking b 
WHERE b.overall_cost = ( SELECT MAX(overall_cost) FROM booking WHERE b.start_date = start_date);"""),
    ("Запрос 9","""SELECT booking_id, start_date, end_date, house_id FROM booking b 
WHERE b.extra_service_id = SOME ( SELECT extra_service_id FROM extra_service WHERE extra_service_id = 5);"""),
    ("Запрос 10","""SELECT b.booking_id, b.overall_cost, 'Confirmed' AS booking_status FROM booking b 
WHERE b.status = 'Подтверждено' AND NOT EXISTS ( SELECT 1 FROM payment p WHERE p.booking_id = b.booking_id AND p.paid = true)
UNION 
SELECT b.booking_id, p.amount, 'Confirmed and paid' AS booking_status FROM booking b JOIN payment p ON b.booking_id = p.booking_id
WHERE b.status = 'Подтверждено' AND p.paid = true ORDER BY booking_id;"""),
    ("Запрос обновление","UPDATE house SET cost_per_day = 3500 WHERE cost_per_day = 4000;")
]


for i, (query_name, query) in enumerate(query_list):
    button = tk.Button(root, text=query_name, command=lambda q=query: run_query(q))
    button.place(x=10+80*i, y=300) 

error_label = tk.Label(root, text="", fg="red")
error_label.grid()

fixed_frame = tk.Frame(root, bg="white", width=500, height=200)
fixed_frame.place(x=10, y=400)

scrollbar_y = ttk.Scrollbar(fixed_frame, orient=tk.VERTICAL)
scrollbar_x = ttk.Scrollbar(fixed_frame, orient=tk.HORIZONTAL)

tree = ttk.Treeview(fixed_frame, yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set, columns=("col1", "col2"), show="headings")

tree.grid(row=0, column=0, sticky=tk.NSEW)
scrollbar_y.grid(row=0, column=1, sticky=tk.NS)
scrollbar_x.grid(row=1, column=0, sticky=tk.EW)

tree.configure(yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)

root.mainloop()